In [8]:
from tools.base import *
from tools.ddmtd import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import sleep 
from copy import deepcopy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#Generate column names
NUM_WORDS = 32
column_names = []
for i in range(1,NUM_WORDS+1):
    column_names.append(f"edge{i}")
    column_names.append(f"ddmtd{i}")
    
pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)

In [10]:
server="trenz_fpga"
rpi_server="trenz_pi"

In [11]:
def get_data(step,channel =5,folder="",aq_iter = 100):
    data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
    !mkdir -p {data_folder}
    !../acq_software/run_atPi.sh ../acq_software/ {step} {channel} {rpi_server}
    !../acq_software/run_atTrenz.sh ../acq_software/ {aq_iter} {server}
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt
        

def plot_data(step=0,channel =10,folder="",freq=160*10**6,data_stream = (1,3),draw=False):   
    df = data2df(step
             ,channel=channel
             ,folder=folder
             ,freq = freq
             ,data_stream = data_stream
            )
#     popt,p_err = df.drawTIE(save_name='',bns=None,fit=True, sep="TIE_FALL")
    popt,p_err = df.drawTIE(save_name='',bns=None,fit=True, sep="",draw=draw)
    


    return (popt[1]*1000,p_err[1]*1000)
    
    

def plot_tie(tie):
    f,ax = plt.subplots(figsize=(8,6))
    
    ax.errorbar(tie[0],tie[1],yerr=tie[2],fmt='o',
#                         color='red',
                        capsize=5,
                        ecolor='black'
                        ,label=f'ODD'
                       )
#     ax.errorbar(tie[0],tie[3],yerr=tie[4],fmt='o',
# #                         color='red',
#                         capsize=5,
#                         ecolor='black'
#                         ,label=f'EVEN'
#                        )
    plt.xlabel("Delay cell index", fontsize=15)
    plt.ylabel("TIE [ps] staggered", fontsize=15)
    plt.legend(loc="upper right")
    plt.gca().grid()
    
def data2df(step=0,channel =5,folder="",freq=160*10**6, data_stream = (1,3)):
    data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
    dv1= pd.read_csv(f"{data_folder}/ddmtd1.txt",sep=",",header=0 ,skiprows=0,names=column_names[0:16])
    dv2= pd.read_csv(f"{data_folder}/ddmtd2.txt",sep=",",header=0 ,skiprows=0,names=column_names[16:32])
    dv3= pd.read_csv(f"{data_folder}/ddmtd3.txt",sep=",",header=0 ,skiprows=0,names=column_names[32:48])
    dv = pd.concat((dv1,dv2,dv3),axis=1)  
#     dv = dv1
#     df = deepcopy(dv[(dv.T != 0).any()])
#     df = df.reset_index(drop=1)
    df = deepcopy(dv)
#     return df 
#     df = df[df.ddmtd3!=2147483647].reset_index(drop=1).drop([0,1,2])    
#     df.iloc[0,:]=0
#     return df
    data = ddmtd(deepcopy(df),q=1,channel=data_stream)
    data.N=100000
    data.INPUT_FREQ = freq
    data.Recalc()
    return data 

def config_dcps(step,channel):
    !../acq_software/run_atPi.sh ../acq_software/ {step} {channel} {rpi_server}


    
def get_data_debug(step,channel =5,folder="",aq_iter = 100,sleep_time=20000):
    data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
    !mkdir -p {data_folder}
    !../acq_software/run_atTrenz.sh ../acq_software/ {aq_iter} {server} {sleep_time}
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
    !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt

In [13]:
freq=160
# !../RPi/config_pll.sh {rpi_server} ../RPi/ 2021_07_20_Dispersion/OCXO/Si5344H-RevD-Registers_{freq}M.h 2021_07_20_Dispersion/PLL/Si5344H-RevD-Registers_{freq}M.h
!../RPi/config_pll.sh {rpi_server} ../RPi/ 2021_07_20_Dispersion/OCXO/Si5344H-RevD-Registers_{freq}M.h 2021_08_09_Si5344H/PLL/Si5344H-RevD-Registers_{freq}M.h





We are in dir::/home/pi/Flash_Firmware
Done Compiling PLLs

TBUFF[2] = 60

RBUFF[0] = 0
RBUFF[1] = 0
RBUFF[2] = 60
address = 0x0514, value = 0x01 ; Value returned = 0x00   
address = 0x001c, value = 0x01 ; Value returned = 0x00   

TBUFF[2] = 47

RBUFF[0] = 0
RBUFF[1] = 0
RBUFF[2] = 47
address = 0x0514, value = 0x01 ; Value returned = 0x00   
address = 0x001c, value = 0x01 ; Value returned = 0x00   


In [16]:
folder_name = f"2021_08_13_{freq}MHz"



In [18]:
get_data(0,channel=5,folder=folder_name,aq_iter=500)

0 delay step selected 
5 channel selected 
Selected Fiber 5
loading firmware: 2021_08_09_DCPS_v2.bit.bin
receiving file list ... 
1 file to consider
ddmtd1.txt
    42503944 100%   10.10MB/s    0:00:04 (xfer#1, to-check=0/1)

sent 38 bytes  received 42514431 bytes  4475207.26 bytes/sec
total size is 42503944  speedup is 1.00
receiving file list ... 
1 file to consider
ddmtd2.txt
    46294701 100%   10.17MB/s    0:00:04 (xfer#1, to-check=0/1)

sent 38 bytes  received 46306108 bytes  8419299.27 bytes/sec
total size is 46294701  speedup is 1.00
receiving file list ... 
1 file to consider
ddmtd3.txt
    46317474 100%   10.48MB/s    0:00:04 (xfer#1, to-check=0/1)

sent 38 bytes  received 46328889 bytes  8423441.27 bytes/sec
total size is 46317474  speedup is 1.00


In [ ]:
tie,tie_err = plot_data(0
             ,channel=5
             ,folder=folder_name
             ,freq = freq*10**6
             ,data_stream = (1,3)
                )